# Retrieval-Augmented Generation: Question Answering based on Custom Dataset with Open-sourced [LangChain](https://python.langchain.com/en/latest/index.html) Library
- 원본 코드
    - https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart-foundation-models/question_answering_retrieval_augmented_generation/question_answering_langchain_jumpstart.ipynb

# 1. 기본 환경 설정

In [101]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../common_code')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
import time
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base


sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
model_version = "*"

## 모델 정보 입력
- SageMaker 엔드포인트 ARN 입력 등

In [103]:
_MODEL_CONFIG_ = {
    "KoAlpaca-12-8B": {
        "instance type": "ml.g5.12xlarge",
        "endpoint_name" : "KoAlpaca-12-8B-2023-05-30-15-03-24",
        "env": {"TS_DEFAULT_WORKERS_PER_MODEL": "1"},
        "parse_function": "parse_response_model_KoAlpaca",
        "prompt": """Answer based on context:\n\n{context}\n\n{question}""",
    },
    "KoSimCSE-roberta": {
        "instance type": "ml.g5.12xlarge",
        "endpoint_name" : "KoSimCSE-roberta-2023-05-30-12-57-53",        
        "env": {"TS_DEFAULT_WORKERS_PER_MODEL": "1"},
    },
}

# 2. LLM 에 Context 없이 추론 테스트

In [104]:
# question = "Which instances can I use with Managed Spot Training in SageMaker?"
q = "홈플러스 중계점은 몇시까지 장사해?"
c = None
# prompt_wo_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
prompt_wo_c = f"### question: {q}\n\n### context: {c}\n\n### answer:" if c else f"### question: {q}\n\n### answer:" 
print("prompt_wo_c: \n", prompt_wo_c)

prompt_wo_c: 
 ### question: 홈플러스 중계점은 몇시까지 장사해?

### answer:


In [105]:

from inference_lib import invoke_inference, query_endpoint_with_text_payload
from inference_lib import parse_response_text_model

model_id = "KoAlpaca-12-8B"
endpoint_name = _MODEL_CONFIG_[model_id]["endpoint_name"]

query_response = query_endpoint_with_text_payload(
    prompt_wo_c, endpoint_name=endpoint_name, 
)

query_response = parse_response_text_model(query_response)
print(query_response)

### question: 홈플러스 중계점은 몇시까지 장사해?

### answer: 안녕하세요.홈플러스 중계점은 밤 11시까지 영업합니다.   
단, 2월은 수요일과 일요일을 제외한 날에 밤 9시에 영업을 마칩니다.   
<2021년 2월>                요일                      
                            
 1일, 6일                       2일, 7일https://www.homeplus.co.kr/company/chedy.do?DM=Y000400300700&gb=Y000004003008000066&source=Y00000400300700https://www.tesat.or.kr/contents/list.asp?board_id=texticon&s_menu_cd=005001001&brd_class=D&contents_id=369                       경기불황으로 호주머니가 얇아진 소비자들의 소비 심리가 위축되면서 저렴하면서도 실질적인 혜택을 제공하는 상품과 서비스가 인기를 끌고 있다. 소비자들은 이제 가격이 조금 비싸더라도 자신에게 알맞은 상품과 서비스를 선택하는 '알뜰한 사치'를 즐기고 있다.

### 답변:소비자는 이제 가격이 조금 비싸더라도 자신에게 알맞은 상품과 서비스를 선택하는 '알뜰한 사치'를 즐기고 있습니다. 이를 실현하기 위해 홈플러스에서는 '한우 국탕거리' 한 품목만 판매하여 매출이 전년 대비 140% 증가했으며, 계절에 따라 할인 행사를 전개해 저렴한 가격에 제공하고 있습니다. 또한, 홈플러스 온라인 전용센터에서는 가격이 조금 비싸더라도 고객이 원하는 제품을 판매하는 전략을 취하여 매출을 6% 증대시켰습니다. 위 두 가지 전략은 홈플러스를 비롯한 여러 기업에서 이용되고 있습니다. 


You can see the generated answer is wrong or doesn't make much sense. 

# 3. 데이터 준비

In [106]:
import glob
import os
import pandas as pd

all_files = glob.glob(os.path.join("../Data/", "amazon_faq_ko.csv"))
# all_files = glob.glob(os.path.join("rag_data/", "Korean_Sample_FAQ.csv"))
# all_files = glob.glob(os.path.join("rag_data/", "English_Sample_FAQ.csv"))



df_knowledge = pd.concat(
    (pd.read_csv(f, names=["Question", "Answer"]) for f in all_files),
    axis=0,
    ignore_index=True,
)

In [107]:
df_knowledge.drop(["Question"], axis=1, inplace=True)

In [108]:
df_knowledge.to_csv("rag_data/amazon_faq_ko_processed_data.csv", header=False, index=False)

## Sample File
- Lang Chain CSV Loader Code
    - https://github.com/hwchase17/langchain/blob/master/langchain/document_loaders/csv_loader.py

In [127]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="rag_data/amazon_faq_ko_processed_data.csv", encoding="utf-8")
documents = loader.load()
documents[0:3]


[Document(page_content='Answer_ko: 아마존 매장에서 상품을 판매하려면 어떻게 해야 하나요?아마존에 등록하면 한 개 또는 수천 개의 아이템을 유연하게 판매할 수 있습니다. 필요에 따라 셀링 플랜을 선택하면 언제든지 플랜을 변경할 수 있습니다.셀러 센트럴을 사용하여 상품 리스팅을 생성하십시오.이 퀵 스타트', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 0}),
 Document(page_content='Answer_ko: 가능성은 사실상 무한합니다.판매할 수 있는 품목은 상품, 상품 카테고리 및 브랜드에 따라 다릅니다.모든 셀러가 이용할 수 있는 카테고리도 있고, 프로페셔널 셀러 계정이 필요한 카테고리도 있고, 판매 승인이 필요한 카테고리도 있고, 타사 셀러가 판매할 수 없는 상품을 포', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 1}),
 Document(page_content='Answer_ko: “일부 상품은 법률 또는 규제 제한 (예: 처방약) 또는 아마존 정책 (예: 범죄 현장 사진) 을 준수하는 것으로 리스팅되지 않을 수 있습니다.자세한 내용은 셀러 센트럴 도움말을 방문하여 제한 사항 및 특정 카테고리의 신상품에 대한 승인을 요청하는 방법에 대해 자세히', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 2})]

# 4 SageMaker Endpoint Wrapper 준비

## SageMaker LLM Wrapper

In [142]:
from langchain.llms.sagemaker_endpoint import SagemakerEndpoint

In [143]:
from inference_lib import KoAlpacaContentHandler
_KoAlpacaContentHandler = KoAlpacaContentHandler()

In [144]:
parameters = {}

sm_llm = SagemakerEndpoint(
    endpoint_name=_MODEL_CONFIG_["KoAlpaca-12-8B"]["endpoint_name"],
    region_name=aws_region,
    model_kwargs=parameters,
    content_handler=_KoAlpacaContentHandler,
)

## SageMaker Embedding Model Wrapper

In [113]:
from inference_lib import SagemakerEndpointEmbeddingsJumpStart
from inference_lib import KoSimCSERobertaContentHandler

In [114]:

_KoSimCSERobertaContentHandler = KoSimCSERobertaContentHandler()

# content_handler = ContentHandler()

embeddings = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=_MODEL_CONFIG_["KoSimCSE-roberta"]["endpoint_name"],
    region_name=aws_region,
    content_handler=_KoSimCSERobertaContentHandler,
)

**Now, we can build an QA application. <span style="color:red">LangChain makes it extremly simple with following few lines of code</span>.**

# 5. Vector Store 생성
- FAISS Vector Store 생성

In [115]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain


In [116]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    text_splitter=CharacterTextSplitter(chunk_size=300, chunk_overlap=0),

)

In [117]:
index = index_creator.from_loaders([loader])

/root/aws-ai-ml-workshop-kr/sagemaker/generative-ai/1-Chatbot/2-Lab02-RAG-LLM/../common_code/inference_lib.py:108: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ndim = np.array(response_json).ndim


In [118]:
index.vectorstore.index_to_docstore_id

{0: '1a792bb9-9574-41ab-b172-a6ac5d1e6023',
 1: 'ca64d9dc-27e8-431a-ba5c-d4170632d2f6',
 2: 'c67ffc28-db4f-4945-bd09-274b159efb96',
 3: '56924664-465b-4895-90a4-a910f31a1c84',
 4: 'b11ee5fb-e942-49d5-b0f8-f1affe0bdeb9',
 5: 'a7c29ab1-21f5-4a98-a1af-bb6b2227a490',
 6: '55908587-7939-4030-bf66-2fe3d8b59139',
 7: 'a5f33df7-722a-4df2-957e-6b239110300b',
 8: '19a3bf96-8e92-4a97-aafc-ca342f51fef6',
 9: '1bbca5c1-dab0-4f45-a9f3-1f0063242692',
 10: '41294e2e-c13b-4349-a1e8-8b7b762ca726',
 11: '8dcf39a0-ec01-4703-a453-1ccb186736f4'}

In [145]:
question = '아마존 스토어에서 무엇을 판매할 수 있나요?'
question2 = 'What items can we sell at Amazon store?'

In [146]:
index.query(question=question2, llm=sm_llm)

input_str: 
 b'{"text_inputs": "Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\\n\\nAnswer_ko: \\uc544\\ub9c8\\uc874 \\ub9e4\\uc7a5\\uc5d0\\uc11c \\uc0c1\\ud488\\uc744 \\ud310\\ub9e4\\ud558\\ub824\\uba74 \\uc5b4\\ub5bb\\uac8c \\ud574\\uc57c \\ud558\\ub098\\uc694?\\uc544\\ub9c8\\uc874\\uc5d0 \\ub4f1\\ub85d\\ud558\\uba74 \\ud55c \\uac1c \\ub610\\ub294 \\uc218\\ucc9c \\uac1c\\uc758 \\uc544\\uc774\\ud15c\\uc744 \\uc720\\uc5f0\\ud558\\uac8c \\ud310\\ub9e4\\ud560 \\uc218 \\uc788\\uc2b5\\ub2c8\\ub2e4. \\ud544\\uc694\\uc5d0 \\ub530\\ub77c \\uc140\\ub9c1 \\ud50c\\ub79c\\uc744 \\uc120\\ud0dd\\ud558\\uba74 \\uc5b8\\uc81c\\ub4e0\\uc9c0 \\ud50c\\ub79c\\uc744 \\ubcc0\\uacbd\\ud560 \\uc218 \\uc788\\uc2b5\\ub2c8\\ub2e4.\\uc140\\ub7ec \\uc13c\\ud2b8\\ub7f4\\uc744 \\uc0ac\\uc6a9\\ud558\\uc5ec \\uc0c1\\ud488 \\ub9ac\\uc2a4\\ud305\\uc744 \\uc0dd\\uc131\\ud558\\uc2ed\\uc2dc\\uc624.\\uc774 

ValueError: Error raised by inference endpoint: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (424) from primary with message "{
  "code":424,
  "message":"prediction failure",
  "error":"The size of tensor a (2048) must match the size of tensor b (2066) at non-singleton dimension 3"
}". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/KoAlpaca-12-8B-2023-05-30-15-03-24 in account 057716757052 for more information.

In [120]:
index.query(question=question, llm=sm_llm)

ValueError: Error raised by inference endpoint: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (424) from primary with message "{
  "code":424,
  "message":"prediction failure",
  "error":"The size of tensor a (2048) must match the size of tensor b (2209) at non-singleton dimension 3"
}". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/KoAlpaca-12-8B-2023-05-30-15-03-24 in account 057716757052 for more information.

# 6. 다른 프로프트로 QA 애플리케이션 테스트

In [121]:
docsearch = FAISS.from_documents(documents, embeddings)

In [122]:
question

'아마존 스토어에서 무엇을 판매할 수 있나요?'

Based on the question above, we then **identify top K most relevant documents based on user query, where K = 3 in this setup**.

In [123]:
docs = docsearch.similarity_search(question, k=3)
docs

[Document(page_content='Answer_ko: 아마존 매장에서 상품을 판매하려면 어떻게 해야 하나요?아마존에 등록하면 한 개 또는 수천 개의 아이템을 유연하게 판매할 수 있습니다. 필요에 따라 셀링 플랜을 선택하면 언제든지 플랜을 변경할 수 있습니다.셀러 센트럴을 사용하여 상품 리스팅을 생성하십시오.이 퀵 스타트', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 0}),
 Document(page_content='Answer_ko: 아마존 매장에서 매출을 늘리는 방법에는 여러 가지가 있습니다.Fulfillment by Amazon (아마존 주문처리 서비스) 을 통해 고객에게 프라임 배송을 제공하여 프라임 회원에게 어필하십시오.또한 다음과 같은 작업을 수행할 수 있습니다.\r\n프로모션, 쿠폰 또는 라', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 11}),
 Document(page_content='Answer_ko: 아마존 스토어에는 신규 판매자에게 많은 기회가 있습니다.판매할 수 있는 품목은 상품, 카테고리 및 브랜드에 따라 다릅니다.모든 셀러가 이용할 수 있는 카테고리도 있고 프로페셔널 셀러 계정이 필요한 카테고리도 있습니다.특정 상품은 판매 승인이 필요하며 다른 카테고리에는', metadata={'source': 'rag_data/amazon_faq_ko_processed_data.csv', 'row': 4})]

Print out the top 3 most relevant docuemnts as below.

Finally, we **combine the retrieved documents with prompt and question and send them into SageMaker LLM.** 

We define a customized prompt as below.

In [124]:
prompt_template = """Answer based on context:\n\n{context}\n\n{question}"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
PROMPT

PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='Answer based on context:\n\n{context}\n\n{question}', template_format='f-string', validate_template=True)

In [125]:
chain = load_qa_chain(llm=sm_llm, prompt=PROMPT)

Send the top 3 most relevant docuemnts and question into LLM to get a answer.

In [126]:
result = chain({"input_documents": docs, "question": question}, return_only_outputs=True)[
    "output_text"
]
result

ValueError: Error raised by inference endpoint: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (424) from primary with message "{
  "code":424,
  "message":"prediction failure",
  "error":"The size of tensor a (2048) must match the size of tensor b (2049) at non-singleton dimension 3"
}". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/KoAlpaca-12-8B-2023-05-30-15-03-24 in account 057716757052 for more information.

Print the final answer from LLM as below, which is accurate.

In [ ]:
r2 = chain({"input_documents": docs, "question": question}, return_only_outputs=True)[
    "output_text"
]
r2